In [15]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

# 鍵とnonceを生成
key = get_random_bytes(32)   # AES-256用
nonce = get_random_bytes(12) # GCM推奨サイズ
out={"original":[],"encrypted":[]}
# 暗号化
for i in [60,70,80,90]:
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    aad = b"header-data"  # 認証のみ対象にする追加データ
    cipher.update(aad)
    original="secret message"*i
    plaintext = (original).encode('utf-8')
    ciphertext, tag = cipher.encrypt_and_digest(plaintext)

    # 暗号化結果をまとめる（Nonce + Ciphertext + Tag）
    encrypted_package = nonce + ciphertext + tag
    print("暗号化パッケージ:", encrypted_package)

    # 復号化
    nonce_dec = encrypted_package[:12]
    ciphertext_dec = encrypted_package[12:-16]
    tag_dec = encrypted_package[-16:]

    cipher_dec = AES.new(key, AES.MODE_GCM, nonce=nonce_dec)
    cipher_dec.update(aad)
    decrypted = cipher_dec.decrypt_and_verify(ciphertext_dec, tag_dec)

    print("復号結果:", decrypted)
    import base64
    encoded_str = base64.b64encode(encrypted_package).decode('utf-8')
    print("Base64文字列:", encoded_str)
    print(len(encoded_str))
    print(len(original))
    out["original"].append(len(original))
    out["encrypted"].append(len(encoded_str))
    

暗号化パッケージ: b'\xdf\xa1u\x85\xd0\x97\xb8o\x985\xb9\xb7y\xf1\xc8\x88\x7f\x9f:jaYA\xe9\xf3\x8f\x1b\x0b:\xe2\x99\x15f\x8b\x91\x9c\x0f\x85<-\xbf\x84\x91\x85\xd2\x0f#\x06*\xc42Z\xc4\x8b\x9bW`A\xf9\xfc\xa4\x8f\x16n/\xdbP\x98\xa7\xfd\x9c\xfe%\xb5\x05\x83q\xa7B/jl\x9c\xec\xddsV\x8e\x04S\xd5=\xf7\x9a\xeek\x8f\x8a\xa5\xc5\x8e\xb1\xc2\x0c\xfch\x1a\x03\x182\xd5\x02\xce\xac0\xc5\xb2\x9f\x87X\'\xe3\x89\xe7\x16?\x07\x17\xc7Ck\xffZQ\x1cK\xabWE\x05\xf7\'`\xd0\x94\xe7Zv\xda\xf9\xe8&\x9e\xed}\xd1\x81h\xfa\x85@\x0b\xac0\xf7Vb\x90\x828|d\x12w\xed\x96\xaa\xc2\xf8mL\xc9\x0f\xe8x\xdd\xce\x80\xe9U\xe7f\x99\x8e\xe1\x1a\x9d\x13\xfd\x06R\xcc4\'\xd3o\xd9\xb8\xbe\xc8K5\x7fm\x87\x1aw\xf3\xa5\xc7\x01\xe3\'F-q\xfa\xb2&\xc2\xde\xe1\xb0\x1c\xc7\xccU\xd5\x07\xd4]n\xbe\xba\xe4%\xde\x01\xad!\xa8\r\x9d\x0f-\xc6\xa0IA\xbf\x0f\x1ac\x88\xf4e\x99\xea 5\xfbI&\xfa\x13\x96\x97L\xba\xd7\x91^5R\xbb\x0c\xaf^#\xf4\xdc\x83z\x9a\t\xe6\xfe\xeb\xf3\xa1\x0e~\x1f\xc8\xde\xdc}\x95B\xa4R\xbb\x12N#\xc9t\xfd\x8d\xab0sH\x82\xe4\x1a\xf4\xb6mW\xffAk\

In [17]:
import pandas as pd

pd.DataFrame(out)


,original,encrypted
0,840,1160
1,980,1344
2,1120,1532
3,1260,1720


In [ ]:
import gzip
import base64
import urllib.parse
import json

# 圧縮したい文字列
tmp={
  "status": "success",
  "timestamp": "2025-11-25T09:09:00Z",
  "user": {
    "id": 12345,
    "name": "Dummy User",
    "email": "dummy@example.com",
    "roles": ["admin", "editor", "viewer"],
    "preferences": {
      "language": "ja",
      "timezone": "Asia/Tokyo",
      "notifications": {
        "email": True,
        "sms": False,
        "push": True
      }
    }
  },
  "data": {
    "items": [
      {
        "id": "A001",
        "title": "sample 1",
        "description": "This is dummy data created for testing JSON responses.",
        "tags": ["test", "dummy", "json"],
        "metrics": {
          "views": 120,
          "likes": 45,
          "comments": 10
        }
      }
    ],
    "summary": {
      "total_items": 2,
      "total_views": 350,
      "total_likes": 112,
      "total_comments": 35
    }
  },
 
}
#text = "これはテスト用の文字列です"
text=json.dumps(tmp, ensure_ascii=False)
# 1. UTF-8バイト列に変換
data = text.encode("utf-8")

# 2. gzip圧縮
compressed = gzip.compress(data)

# 3. Base64 (URL-safe) エンコード
encoded = base64.urlsafe_b64encode(compressed).decode("utf-8")

# 4. クエリパラメータに組み込む
params = {"q": encoded}
print(len(base64.urlsafe_b64encode(text.encode("utf-8"))))
print(len(text))
print(len(encoded))
query_string = urllib.parse.urlencode(params)
url = f"https://example.com/search?{query_string}"
print(url)

892
592
604
https://example.com/search?q=H4sIAFD3JGkC_01RT2vVQBD_Ko89W03yzMGeLHjyoAfrxSKyJPtet81mH9mN-nwEulnRgh4KKlZQvGmhogfxoPjnw0xRP4Yzk0aFkGx-_2Z2ZiWcl751Yn0iXFsUyjlxZiK8NgoJsyA8S7J8LU3XsnwzubBOT3KDRK1TDfIroUv8pNn0fI5oLY0i16XWmOXkOmkQVUbqiuCS4IvqLmZX6mxhDbGNrRS1sCVkaXTNhlJ7y9bbWt3BkJt4XDRqphpVF6xeiUrW81bOud6OHBu_Z2tGNpyW5zbt7tISU1uvZ7qQXtt6cI89-aZVKHCG4JmsHP0tWrd9ynVdh0ApvRxu6xUrt04vLjaSJB1q-4oLQ_8J4keIzyEe_z46PPnyiPlSuaLRC2qAVeEJ9I8hfIC4B_E1xK8QH9K7_wHhLYQXsNdfvnb1CsR30H-G-Ioy8RDeQ3zAyP6vp0cQjk--vfy5fwDhGce9gXAfwvfBz23J-TBbjyvlPmgFdNhx2AnN1Sjf6GKYCo3b8T4TZCq9y7Pm1eK2jKo9s0nXkdNhlmyW7PTWy-rWOJ6MSjMyJk7z5C825qbpP91_8dMch_4H2PMMBJwCAAA%3D


In [6]:
len(text)

884

In [20]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
key = get_random_bytes(32)   # AES-256用
nonce = get_random_bytes(12)
cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
aad = b"header-data"  # 認証のみ対象にする追加データ
cipher.update(aad)
#original="secret message"*i
plaintext =data# (original).encode('utf-8')
ciphertext, tag = cipher.encrypt_and_digest(plaintext)

# 暗号化結果をまとめる（Nonce + Ciphertext + Tag）
encrypted_package = nonce + ciphertext + tag
print("暗号化パッケージ:", encrypted_package)

# 復号化
nonce_dec = encrypted_package[:12]
ciphertext_dec = encrypted_package[12:-16]
tag_dec = encrypted_package[-16:]

cipher_dec = AES.new(key, AES.MODE_GCM, nonce=nonce_dec)
cipher_dec.update(aad)
decrypted = cipher_dec.decrypt_and_verify(ciphertext_dec, tag_dec)

print("復号結果:", decrypted)
import base64
encoded_str = base64.b64encode(encrypted_package).decode('utf-8')
print("Base64文字列:", encoded_str)
print(len(encoded_str))
#print(len(original))
#out["original"].append(len(original))
#out["encrypted"].append(len(encoded_str))

暗号化パッケージ: b'\x86%\xb4\x9f\x897\x84\x0b3\xd2M\x14%Qk\x83^\x1c\xdb\xdb\x0b\xbb\x8e\xc2\xd10q\xb86\xa9v\x9a\x94\xab\xc5\x89\x8e\x94\xe8\x830N\xd2\x8bj\xa49%\xb7\x0b\xbe\xed\x8c\x0f\xe0\t\xb4\x16\xd1\x1dKkV}\'\x8d\x88e\xb8\xe6\x06\xc0gq\xcc +\xdb\xa9e\xf8\x86Sy\xb4\x1d\xbf!\x14k\xe4=\x1a\xe6 u\xb6\x0c%\x9f\xe0xy\x1d\x16\xf9\xebHV\xc4\xbf\xd5\xf39\xc5\x8ek\xd4\x0f\xf3\xe1db\x1d\xa1\xc0Y\xecK\x0b\xd4\x15\xd1\x95\xcb\xf8\xb6\xea\x80E\xc5\xa0T\xa8=\xda\x93\x90\x82x\x8b\xc5\x12\xcf\x8bF\xd6\xe4\x08w\xeb\xc5k0@\x8e\xff\x8f`H\xe2\xa0\xa5\x8b\x81!\xffqYJ\x9e\x91\xe8Ur\xcf\xceFQP+\x8f!%!4\xda\xed\xab\xca\xc5x\xcf\x86\x95x}\xb7#Y\xedr\xa2\xf4\xd3\xedI:8\x99\x11\x89\xec\xf0\x95\x12\x04N\x1a\xd3\x1b\xa4\rR\x92\xef\xc9K\xcb\xc2\xa2\x11G\x95z,\xf3\x8ag\xa9\xbbG$\x16\xd5&\x0e\x8a\x92+}\x0e\xe8\xfa\xac\xee\xe8\x9d7\xeb^\\JXf$H:\xbc]\xd9\xad\xf5)\xd4\xaa\xa3X\xdd\x1b\x7f\x1d\x92\x13\xeb\x1d\x14I\x8b\xbc\xd7\xcf\xc8X+\x14\x98\x83\x06\xb3\x08\x02\xc9\'P\xfd\xad\x05Ks\xd3\x7f\xacIK\x99C\xdc\x8b\xc5s\xd2\xd8\x